In [118]:
import json
import pandas as pd
import requests

from selectolax.parser import HTMLParser
from bs4 import BeautifulSoup

from urllib.parse import urljoin

In [9]:
headers = """accept: */*
accept-encoding: gzip, deflate, br
accept-language: ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7
bx-ajax: true
cookie: _ym_uid=1674666078746037980; _ym_d=1674666078; __ddg1_=NDAgEeNJ8GflD7hHorEs; BITRIX_SM_PROJECT_REGION_ID=21; BITRIX_SID=367H3L7R41; _ym_debug=null; usecookie=accept; _ga=GA1.3.1646809603.1674666078; tmr_lvid=bc5bf1f46b2ccbf438bd0f0b1fabfa1d; tmr_lvidTS=1675015175101; BX_USER_ID=38c70cbf3950a7a6a0dca208ce946056; reg-closed=accept; pickup_popup=0; private=0; alco_agreement=no; _gid=GA1.2.201519798.1676993974; _ym_visorc=b; _ym_isad=2; BITRIX_SM_SALE_UID=38d95e86000c81780974f364cbfbef47; BITRIX_CONVERSION_CONTEXT_s1=%7B%22ID%22%3A4%2C%22EXPIRE%22%3A1677013140%2C%22UNIQUE%22%3A%5B%22conversion_visit_day%22%5D%7D; _gid=GA1.3.201519798.1676993974; PHPSESSID=323399f438bc1eb29fd47e9a7cd3d516; is_spb=1; store_id=2373; address=%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3%2C%20%D0%BF%D1%80%D0%BE%D1%81%D0%BF%D0%B5%D0%BA%D1%82%20%D0%A5%D1%83%D0%B4%D0%BE%D0%B6%D0%BD%D0%B8%D0%BA%D0%BE%D0%B2%20; coords=60.044859%2C30.351055; distance=815; price_id=82; alco_allow=0; is_m=false; countmobile=9; LOYALTYCARD=4; _dc_gtm_UA-172001173-1=1; _gat_UA-172001173-1=1; _ga=GA1.1.1646809603.1674666078; _ga_J3JT2KMN08=GS1.1.1676993989.40.1.1676994535.51.0.0; tmr_detect=0%7C1676994535754
referer: https://dostavka.dixy.ru/catalog/voda_soki_napitki_1/?PAGEN_1=2
sec-ch-ua: "Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"
sec-ch-ua-mobile: ?0
sec-ch-ua-platform: "macOS"
sec-fetch-dest: empty
sec-fetch-mode: cors
sec-fetch-site: same-origin
user-agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"""

In [10]:
def parse_header(raw_header: str):
    header = dict()

    for line in raw_header.split("\n"):

        if line.startswith(":"):
            a, b = line[1:].split(":", 1)
            a = f":{a}"
        else:
            a, b = line.split(":",1)

        header[a.strip()] = b.strip()

    return header

In [11]:
headers = parse_header(headers)

In [107]:
resp = requests.get(
    "https://dostavka.dixy.ru/catalog/sales/bakaleya-new/"
)

In [108]:
dom = HTMLParser(resp.text)
soup = BeautifulSoup(resp.text)

In [120]:
{
    el.css_first("span").text(): el.attributes["href"]
    for el in dom.css("div.section-compact-list > div > div > a")
}

{'Бакалея': '/catalog/sales/bakaleya-new/',
 'Молочные продукты, яйца': '/catalog/sales/molochnye-produkty-yaytsa-new/',
 'Овощи, фрукты, зелень, грибы': '/catalog/sales/ovoshchi-frukty-zelen-griby-new/',
 'Товары для детей': '/catalog/sales/tovary-dlya-detey-new/',
 'Детское питание': '/catalog/sales/detskoe-pitanie-new/',
 'Хлеб, выпечка, сладости': '/catalog/sales/khleb-vypechka-sladosti-new/',
 'Рыба, морепродукты, икра': '/catalog/sales/ryba-moreprodukty-ikra-new/',
 'Вода, соки, напитки': '/catalog/sales/voda-soki-napitki-new/',
 'Замороженные продукты': '/catalog/sales/zamorozhennye-produkty-new/',
 'Соусы, специи': '/catalog/sales/sousy-spetsii-new/',
 'Бытовая химия': '/catalog/sales/bytovaya-khimiya-new/',
 'Консервы': '/catalog/sales/konservy-new/',
 'Сыры': '/catalog/sales/syry-new/'}

In [119]:
urljoin("https://dostavka.dixy.ru/catalog/sales/", "/catalog/sales/bakaleya-new/")

'https://dostavka.dixy.ru/catalog/sales/bakaleya-new/'

In [113]:
dom.css_first("div.nums a.dark_link:last-child").text()

'19'

In [84]:
len(dom.css("div.catalog_block > div.item_block"))

20

In [150]:
df_1 = pd.read_json("/Users/borokoko/Crawlers/data/Diksi/diksi_1.json")
df_2 = pd.read_json("/Users/borokoko/Crawlers/data/Diksi/diksi_2.json")

In [151]:
df_1 = df_1.drop_duplicates(subset="name")
df_2 = df_2.drop_duplicates(subset="name")

# df_1 = df_1[df_1.price_cur.notna() & df_1.price_prev.notna()]
# df_2 = df_2[df_2.price_cur.notna() & df_2.price_prev.notna()]

In [152]:
df_m = df_1.merge(df_2, on="name", how="inner")

In [156]:
df_m = df_m[df_m.price_cur_x.notna() & df_m.price_cur_y.notna()]

df_m[df_m.price_cur_x != df_m.price_cur_y]

,name,price_cur_x,category_x,subcategory_x,price_prev_x,price_prev_y,price_cur_y,category_y,subcategory_y
11062,"Набор для творчества Колорит Гравюра большая, ...",99.99,Товары для детей,Игрушки и творчество,NaN,99.99,19.99,Товары для дома и дачи,Канцелярские товары


In [133]:
df = df.drop_duplicates(subset="name")

In [135]:
df[df.price_cur.notna() & df.price_prev.notna()]

,name,price_cur,category,subcategory,price_prev
1,"Средство защиты от мух ""Капкан"" Липкая лента ,...",79.99,Товары для дома и дачи,Товары для садоводства,119.90
5,"Грунт универсальный торфяной, 10л",9.99,Товары для дома и дачи,Товары для садоводства,49.99
6,"Наклейки для маркировки рассады СЗТК, 40шт",9.99,Товары для дома и дачи,Товары для садоводства,39.99
8,"Перчатки трикотажные с ПВХ-покрытием ""Точка"", ...",9.99,Товары для дома и дачи,Товары для садоводства,90.00
9,"Почвогрунт ""ЭКЗО"" Универсальный, 10л",55.99,Товары для дома и дачи,Товары для садоводства,79.99
...,...,...,...,...,...
20945,"Десерт творожный взбитый ""Чудо"" Персик-Маракуй...",49.99,"Молочные продукты, яйца",Йогурты,51.49
20956,"Десерт творожный взбитый ""Чудо"" Малина-Ежевика...",49.99,"Молочные продукты, яйца",Йогурты,51.99
21145,"Творожок ""Даниссимо"" Пломбир, 110г",44.99,"Молочные продукты, яйца",Йогурты,46.49
21346,"Продукт творожный ""Даниссимо"" Крем-брюле, 5,5%...",44.99,"Молочные продукты, яйца",Йогурты,46.99
